In [2]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import matplotlib.ticker as mticker
import seaborn as sns

In [3]:
df = pd.read_csv('Motor_Vehicle_Collisions.csv')

C:\Users\alvar\AppData\Local\Temp\ipykernel_11136\391125704.py:1: DtypeWarning: Columns (3) have mixed types. Specify dtype option on import or set low_memory=False.
  df = pd.read_csv('Motor_Vehicle_Collisions.csv')


# **Feature Selection**

In [4]:
# For sure, these columns will be of no use
columns_to_drop = [
    'ZIP CODE',
    'ON STREET NAME',
    'CROSS STREET NAME',
    'OFF STREET NAME',
    'COLLISION_ID',
    'CONTRIBUTING FACTOR VEHICLE 2',
    'CONTRIBUTING FACTOR VEHICLE 3',
    'CONTRIBUTING FACTOR VEHICLE 4',
    'CONTRIBUTING FACTOR VEHICLE 5',
    'VEHICLE TYPE CODE 1',
    'VEHICLE TYPE CODE 2',
    'VEHICLE TYPE CODE 3',
    'VEHICLE TYPE CODE 4',
    'VEHICLE TYPE CODE 5'
]

# Drop the columns
df = df.drop(columns=columns_to_drop)

# **Missing Values: BOROUGH**

First, let's delete the observation with no LOCATION. They are a very small percentage of the total number of accidents (all below 5% per year)

In [5]:
df = df.dropna(subset=['LOCATION'])
# Verify the result
print("Number of rows with null LOCATION after dropping:", df['LOCATION'].isnull().sum())

Number of rows with null LOCATION after dropping: 0


Obtain:
- missing_location_df: those observations that have missing BOROUGH but not missing LOCATION.
- training_df: those observation to train the KNN with. The don't have NaN BOROUGH nor LOCATION.

In [6]:
df['LOCATION'].isnull().sum()

# Data set with the NaN location values
missing_location_df = df[(df['BOROUGH'].isnull())&(df['LOCATION'].notnull())]

# Data set we will use to train our KNN model
training_df = df[(df['BOROUGH'].notnull())&(df['LOCATION'].notnull())]

# Display the shapes to verify (optional)
print(f"Original df shape: {df.shape}")
print(f"Missing Borough df shape: {missing_location_df.shape}")
print(f"Training df shape: {training_df.shape}")

Original df shape: (1929241, 15)
Missing Borough df shape: (468030, 15)
Training df shape: (1461211, 15)


## **KNN**

Standardize/scale the data

In [7]:
from sklearn.preprocessing import StandardScaler

# Extract features for training
X = training_df[['LATITUDE', 'LONGITUDE']]
y = training_df['BOROUGH']  # Target variable

# Standardize features
scaler = StandardScaler()
X_scaled = scaler.fit_transform(X)

# For prediction data
X_pred = missing_location_df[['LATITUDE', 'LONGITUDE']]
X_pred_scaled = scaler.transform(X_pred)

Hold-out method for validation: 80% train, 20% validation

In [8]:
from sklearn.neighbors import KNeighborsClassifier
from sklearn.model_selection import train_test_split
from sklearn.metrics import classification_report, accuracy_score

# Split the data
X_train, X_test, y_train, y_test = train_test_split(X_scaled, y, test_size=0.2, random_state=17)

# Initialize and train the KNN classifier
knn = KNeighborsClassifier(n_neighbors=5)
knn.fit(X_train, y_train)

# Make predictions on test set
y_pred = knn.predict(X_test)

# Evaluate the model
print("Model Accuracy:", accuracy_score(y_test, y_pred))
print("\nDetailed Classification Report:")
print(classification_report(y_test, y_pred))

Model Accuracy: 0.9973173010131979

Detailed Classification Report:
               precision    recall  f1-score   support

        BRONX       1.00      1.00      1.00     43321
     BROOKLYN       0.99      1.00      1.00     93560
    MANHATTAN       1.00      1.00      1.00     64606
       QUEENS       1.00      1.00      1.00     78516
STATEN ISLAND       1.00      1.00      1.00     12240

     accuracy                           1.00    292243
    macro avg       1.00      1.00      1.00    292243
 weighted avg       1.00      1.00      1.00    292243



In [9]:
# Create a proper copy of the DataFrame for predictions
missing_location_df_copy = missing_location_df.copy()

# Now predict the boroughs for the missing values
missing_borough_predictions = knn.predict(X_pred_scaled)

# Add predictions to the copy
missing_location_df_copy.loc[:, 'PREDICTED_BOROUGH'] = missing_borough_predictions

# Optional: Display the first few predictions
print("\nFirst few predictions for missing boroughs:")
print(missing_location_df_copy[['LATITUDE', 'LONGITUDE', 'PREDICTED_BOROUGH']].head())

# Fill the rows with the imputed BOROUGH values
df.loc[missing_location_df.index, 'BOROUGH'] = missing_location_df_copy['PREDICTED_BOROUGH']


First few predictions for missing boroughs:
     LATITUDE  LONGITUDE PREDICTED_BOROUGH
12  40.709183 -73.956825          BROOKLYN
16  40.701275 -73.888870            QUEENS
19  40.596620 -74.002310          BROOKLYN
25  40.783268 -73.824850            QUEENS
27  40.744644 -73.770410            QUEENS


------------------------------------------------------
Drop the rest of rows with NaN values (victims features)

In [10]:
df = df.dropna(subset=['NUMBER OF PERSONS INJURED','NUMBER OF PERSONS KILLED','CONTRIBUTING FACTOR VEHICLE 1', ])

# **Preprocessing: CONTRIBUTING FACTOR**

In [11]:
len(df)

1922359

In [12]:
df['CONTRIBUTING FACTOR VEHICLE 1'].unique()

array(['Unspecified', 'Passing Too Closely', 'Driver Inexperience',
       'Passing or Lane Usage Improper', 'Turning Improperly',
       'Unsafe Speed', 'Reaction to Uninvolved Vehicle',
       'Steering Failure', 'Following Too Closely', 'Other Vehicular',
       'Driver Inattention/Distraction', 'Oversized Vehicle',
       'Traffic Control Disregarded', 'Unsafe Lane Changing',
       'Alcohol Involvement', 'View Obstructed/Limited',
       'Failure to Yield Right-of-Way', 'Aggressive Driving/Road Rage',
       'Pavement Slippery', 'Illnes', 'Lost Consciousness',
       'Brakes Defective', 'Backing Unsafely', 'Passenger Distraction',
       'Fell Asleep',
       'Pedestrian/Bicyclist/Other Pedestrian Error/Confusion',
       'Obstruction/Debris', 'Tinted Windows', 'Animals Action',
       'Drugs (illegal)', 'Pavement Defective', 'Other Lighting Defects',
       'Outside Car Distraction', 'Driverless/Runaway Vehicle',
       'Tire Failure/Inadequate', 'Fatigued/Drowsy',
       'Headli

In [13]:
(df['CONTRIBUTING FACTOR VEHICLE 1']=='1').sum()

5

In [14]:
df_new = df.copy()

# Create a mapping dictionary to group similar contributing factors
contributing_factor_mapping = {
    # Driver Behavior - Attention
    'Driver Inattention/Distraction': 'Driver Attention Issues',
    'Passenger Distraction': 'Driver Attention Issues',
    'Outside Car Distraction': 'Driver Attention Issues',
    'Cell Phone (hands-free)': 'Driver Attention Issues',
    'Cell Phone (hand-Held)': 'Driver Attention Issues',
    'Cell Phone (hand-held)': 'Driver Attention Issues',
    'Using On Board Navigation Device': 'Driver Attention Issues',
    'Other Electronic Device': 'Driver Attention Issues',
    'Listening/Using Headphones': 'Driver Attention Issues',
    'Texting': 'Driver Attention Issues',
    'Eating or Drinking': 'Driver Attention Issues',
    
    # Driver Behavior - Impairment
    'Alcohol Involvement': 'Driver Impairment',
    'Drugs (illegal)': 'Driver Impairment',
    'Drugs (Illegal)': 'Driver Impairment',
    'Fell Asleep': 'Driver Impairment',
    'Fatigued/Drowsy': 'Driver Impairment',
    'Lost Consciousness': 'Driver Impairment',
    'Prescription Medication': 'Driver Impairment',
    
    # Driver Behavior - Aggression/Violations
    'Unsafe Speed': 'Traffic Violations',
    'Following Too Closely': 'Traffic Violations',
    'Passing Too Closely': 'Traffic Violations',
    'Traffic Control Disregarded': 'Traffic Violations',
    'Failure to Yield Right-of-Way': 'Traffic Violations',
    'Aggressive Driving/Road Rage': 'Traffic Violations',
    'Failure to Keep Right': 'Traffic Violations',
    'Backing Unsafely': 'Traffic Violations',
    
    # Driver Skill/Experience
    'Driver Inexperience': 'Driver Skill Issues',
    'Turning Improperly': 'Driver Skill Issues',
    'Passing or Lane Usage Improper': 'Driver Skill Issues',
    'Unsafe Lane Changing': 'Driver Skill Issues',
    
    # Vehicle Issues
    'Steering Failure': 'Vehicle Mechanical Failure',
    'Brakes Defective': 'Vehicle Mechanical Failure',
    'Tire Failure/Inadequate': 'Vehicle Mechanical Failure',
    'Headlights Defective': 'Vehicle Mechanical Failure',
    'Accelerator Defective': 'Vehicle Mechanical Failure',
    'Tow Hitch Defective': 'Vehicle Mechanical Failure',
    'Windshield Inadequate': 'Vehicle Mechanical Failure',
    'Other Lighting Defects': 'Vehicle Mechanical Failure',
    'Oversized Vehicle': 'Vehicle Issues',
    'Driverless/Runaway Vehicle': 'Vehicle Issues',
    'Vehicle Vandalism': 'Vehicle Issues',
    'Tinted Windows': 'Vehicle Issues',
    
    # Environmental Factors
    'Pavement Slippery': 'Road Infrastructure',
    'Pavement Defective': 'Road Infrastructure',
    'View Obstructed/Limited': 'Visibility Issues',
    'Glare': 'Visibility Issues',
    'Obstruction/Debris': 'Road Infrastructure',
    'Lane Marking Improper/Inadequate': 'Road Infrastructure',
    'Shoulders Defective/Improper': 'Road Infrastructure',
    'Traffic Control Device Improper/Non-Working': 'Road Infrastructure',
    
    # Health/Physical Issues
    'Illnes': 'Health Issues',
    'Illness': 'Health Issues',
    'Physical Disability': 'Health Issues',
    
    # External Factors
    'Pedestrian/Bicyclist/Other Pedestrian Error/Confusion': 'External Factors',
    'Animals Action': 'External Factors',
    'Other Vehicular': 'External Factors',
    'Reaction to Uninvolved Vehicle': 'External Factors',
    'Reaction to Other Uninvolved Vehicle': 'External Factors',
    
    # Other/Unspecified
    'Unspecified': 'Unspecified',
    '80': 'Unspecified',  # Likely data entry errors
    '1': 'Unspecified'    # Likely data entry errors
}

# Apply the mapping to create a new column with grouped factors
df_new['CONTRIBUTING_FACTOR_GROUP'] = df_new['CONTRIBUTING FACTOR VEHICLE 1'].map(contributing_factor_mapping)

# Handle any unmapped values (if any new categories appear)
unmapped = df_new[df_new['CONTRIBUTING_FACTOR_GROUP'].isna()]['CONTRIBUTING FACTOR VEHICLE 1'].unique()
if len(unmapped) > 0:
    print("Unmapped contributing factors:", unmapped)
    # Assign unmapped values to 'Other' category
    ddf_newf['CONTRIBUTING_FACTOR_GROUP'] = df_new['CONTRIBUTING_FACTOR_GROUP'].fillna('Other')

# Count the frequency of each group
factor_group_counts = df_new['CONTRIBUTING_FACTOR_GROUP'].value_counts()
print(factor_group_counts)

CONTRIBUTING_FACTOR_GROUP
Unspecified                   645415
Traffic Violations            420333
Driver Attention Issues       417846
Driver Skill Issues           169923
Driver Impairment              97353
External Factors               91155
Road Infrastructure            25288
Visibility Issues              17223
Vehicle Issues                 13144
Vehicle Mechanical Failure     13043
Health Issues                  11636
Name: count, dtype: int64


In [15]:
# Convert CRASH DATE to datetime
df_new['CRASH DATE'] = pd.to_datetime(df_new['CRASH DATE'], format='%m/%d/%Y')

# Sort the DataFrame by CRASH DATE
df_new = df_new.sort_values('CRASH DATE')

# Verify the conversion
print("Data type of CRASH DATE column:", df_new['CRASH DATE'].dtype)
print("\nSample of converted dates:")
print(df_new['CRASH DATE'].head())

Data type of CRASH DATE column: datetime64[ns]

Sample of converted dates:
2101610   2012-07-01
2102086   2012-07-01
2102092   2012-07-01
2102103   2012-07-01
2102111   2012-07-01
Name: CRASH DATE, dtype: datetime64[ns]


In [16]:
df_new['CRASH DATE'].dt.month

2101610    7
2102086    7
2102092    7
2102103    7
2102111    7
          ..
2168948    4
2168949    4
2162987    4
2168937    4
2169086    4
Name: CRASH DATE, Length: 1922359, dtype: int32

In [17]:
df_weather = pd.read_csv('weather_means_2016_2022.csv')

In [18]:
df_weather.columns

Index(['Unnamed: 0', 'date', 'temperature_2m (°C)', 'precipitation (mm)',
       'rain (mm)', 'cloudcover (%)', 'cloudcover_low (%)',
       'cloudcover_mid (%)', 'cloudcover_high (%)', 'windspeed_10m (km/h)',
       'winddirection_10m (°)'],
      dtype='object')

In [19]:
df_weather = df_weather.drop(columns=['Unnamed: 0'])

In [20]:
df_weather

,date,temperature_2m (°C),precipitation (mm),rain (mm),cloudcover (%),cloudcover_low (%),cloudcover_mid (%),cloudcover_high (%),windspeed_10m (km/h),winddirection_10m (°)
0,2016-01-01,5.41,0.0,0.0,57.29,34.50,9.71,77.75,12.48,273.33
1,2016-01-02,2.39,0.0,0.0,10.25,9.92,1.42,1.50,13.38,266.42
2,2016-01-03,3.01,0.0,0.0,10.29,11.29,0.08,0.00,12.75,244.38
3,2016-01-04,0.10,0.0,0.0,14.75,11.83,6.75,0.00,16.85,313.54
4,2016-01-05,-6.78,0.0,0.0,0.67,0.75,0.00,0.00,17.65,304.25
...,...,...,...,...,...,...,...,...,...,...
2187,2021-12-27,2.78,0.0,0.0,45.79,12.67,45.54,40.12,7.55,218.88
2188,2021-12-28,4.30,2.2,2.2,76.17,46.79,64.38,54.25,7.89,220.96
2189,2021-12-29,5.85,5.1,5.1,94.33,72.29,51.29,59.00,6.28,64.38
2190,2021-12-30,6.95,2.4,2.4,100.00,92.67,86.88,50.92,4.31,156.29


# MERGE

In [21]:
# Check if 'date' is in df_weather columns
if 'date' in df_weather.columns:
    # If 'date' is a column in df_weather
    print("Using column merge with 'date' column")
    
    # Ensure both date columns are the same type
    df_new['merge_date'] = pd.to_datetime(df_new['CRASH DATE']).dt.date
    df_weather['date'] = pd.to_datetime(df_weather['date']).dt.date

    # Merge using columns
    df_merged = pd.merge(
        df_new,
        df_weather,
        left_on='merge_date',
        right_on='date',
        how='left'
    )
else:
    # If date is the index in df_weather
    print("Using index merge with date index")
    
    # Convert index to date objects if it's datetime
    if pd.api.types.is_datetime64_any_dtype(df_weather.index):
        df_weather.index = df_weather.index.date
    
    # Convert merge_date to same type as df_weather index
    df_new['merge_date'] = pd.to_datetime(df_new['CRASH DATE']).dt.date
    
    # Merge using index
    df_merged = pd.merge(
        df_new,
        df_weather,
        left_on='merge_date',
        right_index=True,
        how='left'
    )

# Check the result
print("Merge successful?", "Yes" if len(df_merged) > 0 else "No")
print("Original df_new shape:", df_new.shape)
print("Merged DataFrame shape:", df_merged.shape)

# Check for missing values in the first weather column
if len(df_weather.columns) > 0:
    first_weather_col = df_weather.columns[0]
    missing_count = df_merged[first_weather_col].isna().sum()
    print(f"Rows with missing weather data ({first_weather_col}): {missing_count}")

Using column merge with 'date' column
Merge successful? Yes
Original df_new shape: (1922359, 17)
Merged DataFrame shape: (1922359, 27)
Rows with missing weather data (date): 899951


In [22]:
df_merged.columns

Index(['CRASH DATE', 'CRASH TIME', 'BOROUGH', 'LATITUDE', 'LONGITUDE',
       'LOCATION', 'NUMBER OF PERSONS INJURED', 'NUMBER OF PERSONS KILLED',
       'NUMBER OF PEDESTRIANS INJURED', 'NUMBER OF PEDESTRIANS KILLED',
       'NUMBER OF CYCLIST INJURED', 'NUMBER OF CYCLIST KILLED',
       'NUMBER OF MOTORIST INJURED', 'NUMBER OF MOTORIST KILLED',
       'CONTRIBUTING FACTOR VEHICLE 1', 'CONTRIBUTING_FACTOR_GROUP',
       'merge_date', 'date', 'temperature_2m (°C)', 'precipitation (mm)',
       'rain (mm)', 'cloudcover (%)', 'cloudcover_low (%)',
       'cloudcover_mid (%)', 'cloudcover_high (%)', 'windspeed_10m (km/h)',
       'winddirection_10m (°)'],
      dtype='object')

In [23]:
columns_to_drop = ['merge_date', 'date']
df_merged = df_merged.drop(columns=columns_to_drop)

In [24]:
df_merged.columns

Index(['CRASH DATE', 'CRASH TIME', 'BOROUGH', 'LATITUDE', 'LONGITUDE',
       'LOCATION', 'NUMBER OF PERSONS INJURED', 'NUMBER OF PERSONS KILLED',
       'NUMBER OF PEDESTRIANS INJURED', 'NUMBER OF PEDESTRIANS KILLED',
       'NUMBER OF CYCLIST INJURED', 'NUMBER OF CYCLIST KILLED',
       'NUMBER OF MOTORIST INJURED', 'NUMBER OF MOTORIST KILLED',
       'CONTRIBUTING FACTOR VEHICLE 1', 'CONTRIBUTING_FACTOR_GROUP',
       'temperature_2m (°C)', 'precipitation (mm)', 'rain (mm)',
       'cloudcover (%)', 'cloudcover_low (%)', 'cloudcover_mid (%)',
       'cloudcover_high (%)', 'windspeed_10m (km/h)', 'winddirection_10m (°)'],
      dtype='object')

Get only the years where we have data with weather

In [25]:
# df_merged = df_merged[(df_merged['CRASH DATE'].dt.year>=2016)&(df_merged['CRASH DATE'].dt.year<=2021)]

In [26]:
df_merged.to_csv('merged_collisions_weather_2012-2024.csv')